In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

## DATA

In [ ]:
def read_csv_files(folder_path, file_names):
    dataframes = {}
    for file_name in file_names:
        # Construir la ruta completa del archivo
        csv_path = os.path.join(folder_path, file_name)
        
        # Verificar si el archivo existe en la ruta especificada
        if os.path.exists(csv_path):
            # Leer el archivo CSV
            df = pd.read_csv(csv_path)            
            # Añadir el DataFrame al diccionario
            dataframes[file_name] = df
            
            # Mostrar las primeras filas del DataFrame para verificar
            print(f"DataFrame de {file_name}:")
            
        else:
            print(f"El archivo {csv_path} no se encuentra en la ruta especificada.")
    
    return dataframes


folder_path = r"C:\Users\Sofia\Documentos\IMPORTANTES\Estudio\ML-Templates\Classification\Binary_Classification"
file_names = ["gender_submission.csv","test.csv"]
dataframes = read_csv_files(folder_path, file_names)


test = dataframes["test.csv"]
test